### This notebook deletes chosen recordings and all related sounds and traces

In [1]:
cd /home/nakedmoleratvoices/Mole\ rats\ reborn/CodeRefactoredFinal/

/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal


In [2]:
import pandas as pd
import numpy as np
from src.preprocessing.filters import filter_recs_metadata
import os

In [3]:
# load metadata
path_to_recordings_metadata = '/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal/data/'
recordings_metadata_name = 'recordings_metadata.csv'
recs_metadata = pd.read_csv(path_to_recordings_metadata + recordings_metadata_name,)


In [4]:
# set filters for recs to delete
colonies = ['lannister']
dates = 'all'
experiments = 'all'
stages = ['traced and checked',]

filters_mask = filter_recs_metadata(recs_metadata, dates = dates, colonies = colonies, 
                                    stages = stages, experiments = experiments)
# or set their names directly 
# comment 2 lines below to not use them

rec_names = ['baratheon_20-04-20_4020_0000024.npy']
filters_mask = recs_metadata['name'].isin(rec_names)

print(recs_metadata[filters_mask].shape)
recs_metadata[filters_mask].head()

(1, 8)


/usr/local/lib/python3.5/dist-packages/pandas/core/computation/expressions.py:193: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  op=op_str, alt_op=unsupported[op_str]


,colony,date,experiment,name,number,path,processing stage,ratids
1034,baratheon,20-04-20,single rat softchirps;baratheon epochs,baratheon_20-04-20_4020_0000024.npy,24,/home/nakedmoleratvoices/Mole rats reborn/Code...,traced and checked,4020


In [7]:
inds_to_delete = np.where(filters_mask)[0]
for ind in inds_to_delete:
    rec_name = recs_metadata['name'].iloc[ind]
    rec_path = recs_metadata['path'].iloc[ind]
    rec_stage = recs_metadata['processing stage'].iloc[ind]
    print(rec_name, 'is', rec_stage)
    if rec_name in os.listdir(rec_path):
        os.remove(rec_path + rec_name)
        print('deleted rec file')
    else:
        print('Recording was not found')
    
    if rec_stage != 'fresh':
        if rec_path + rec_name[:-3] + 'txt' in os.listdir(rec_path):
            os.remove(rec_path + rec_name[:-3] + 'txt')
            print('deleted txt')
        else:
            print('Txt was not found')
    if 'traced' in rec_stage:
        rec_traces = [t for t in os.listdir(rec_path.replace('recordings', 'traces'))
                     if rec_name[:-3] in t]
        n_traces = len(rec_traces)
        for t in rec_traces:
            os.remove(rec_path.replace('recordings', 'traces') + t)
        print('deleted %d traces' % n_traces)
        
    recs_metadata['processing stage'].iloc[ind] = 'to delete'
    
print('Run the cell below to update metadata file')

baratheon_20-04-20_4020_0000024.npy is traced and checked
Recording was not found
Txt was not found
deleted 0 traces
Run the cell below to update metadata file


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
recs_metadata = recs_metadata.drop(inds_to_delete, axis = 0)
recs_metadata.to_csv(path_to_recordings_metadata + recordings_metadata_name, index = None)